In [85]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from collections import defaultdict
import spacy
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn import metrics
import optuna
from sklearn.svm import SVC


In [ ]:
'''
df = pd.read_csv('factnews_dataset.csv') 

#criar grupo treino, validação e teste
train_df, test_df = train_test_split(df, test_size=0.7, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['classe'])

#guardar conjuntos em memória
train_df.to_csv("train.csv", index=False) 
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)

'''



# Pré-processamento

In [86]:
#carregar conjuntos de treino e teste
train = pd.read_csv('train.csv')
val   = pd.read_csv('val.csv')
test = pd.read_csv('test.csv')

stop_words = set(stopwords.words('portuguese'))

#criar coluna tokens
train['tokens'] = train['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
val['tokens']   = val['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
test['tokens'] = test['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))

#remover stopwords
train['tokens'] = train['tokens'].apply(lambda toks: [t for t in toks if t not in stop_words])
val['tokens']   = val['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
test['tokens']  = test['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])

#matizar
#nltk . download ('rslp')
stemmer = nltk . stem . RSLPStemmer ()
train['stems'] = train['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
val['stems']   = val['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
test['stems'] = test['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

nltk . download ('rslp')
stemmer = nltk . stem . RSLPStemmer ()
train['stems'] = train['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
print(train['stems'].head())

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\didia\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


0    [``, sua, mer, exist, surpr, orig, futur, mist...
1    [segund, ,, companh, pag, hotel, são, paul, fa...
2                  [pf, prend, 25, suspeit, lav, dinh]
3    [não, hav, regr, agor, regr, ríg, ,, ach, dev,...
4    [segund, assessor, lul, ,, classific, manifest...
Name: stems, dtype: object


Análise exploratória

# Construção dos modelos de representação

## TF-IDF - fred

In [ ]:
def TF_IDF():
    tf_vect = TfidfVectorizer ( ngram_range =(1 ,2) , min_df =3 ,
    max_df =0.5 , max_features =500)
    p_tf = tf_vect . fit_transform ( docs )

## CountVectorizer -didi

In [87]:
def count_vec(train,val,test):
    c_vect = CountVectorizer(ngram_range=(1,1), strip_accents='unicode',max_features=1000)

    #juntar os tokens matizados em frases novvamente 
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val   = val['stems'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['stems'].apply(lambda tokens: ' '.join(tokens))
    
    #print("Text train:\n", text_train.head())

    # Transformar em vetores
    X_train = c_vect.fit_transform(text_train)
    X_val   = c_vect.transform(text_val)
    X_test  = c_vect.transform(text_test)

    return X_train, X_val, X_test

In [88]:
X_train_count_vec, X_val_count_vec, X_test_count_vec = count_vec(train,val,test)

## Glove -fred

## WORD2VEC -anaana

fontes : 
-   https://sites.google.com/view/nilc-usp/resources-and-tools?authuser=0
-   https://huggingface.co/nilc-nlp/word2vec-cbow-300d

In [89]:
#%pip install huggingface_hub
#%pip install hf_hub_download
#%pip install safetensors
from huggingface_hub import hf_hub_download
from safetensors.numpy import load_file
from gensim.models import KeyedVectors
def build_word2vec_model():
    #descarregar os embeddings do modelo Word2Vec CBOW 300d do NILC
    path = hf_hub_download(repo_id="nilc-nlp/word2vec-cbow-300d",
                           filename="embeddings.safetensors")   
    
    #carregar os vetores do ficheiro safetensors
    data = load_file(path)
    vectors = data["embeddings"]    

    #descarregar vocabulário
    vocab_path = hf_hub_download(repo_id="nilc-nlp/word2vec-cbow-300d",
                                 filename="vocab.txt")
    
    with open(vocab_path) as f:
        vocab = [w.strip() for w in f]  

    #print(vectors.shape)
    #print(vectors)

    model = KeyedVectors(vector_size=300)
    model.add_vectors(vocab, vectors)

    print(f"Número de palavras no modelo: {len(model.key_to_index)}")
    print(f"Dimensão dos vetores: {model.vector_size}")

In [ ]:
'''from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
certos = (test_df['classe'] == test_df['Previsto4']).sum()
total = len(test_df)

matriz = confusion_matrix(test_df['classe'], test_df['Previsto4'], labels=classes)

disp = ConfusionMatrixDisplay(confusion_matrix=matriz, display_labels=classes)
disp.plot()
plt.title("Matriz de confusão do teste")
plt.show()

from sklearn.metrics import classification_report

report = classification_report(test_df['classe'], test_df['Previsto4'], labels=classes, digits=4)
print(report)'''

# Aplicar modelos de classificação

In [90]:
y_test= test['classe']
y_train= train['classe']
y_val=val['classe']
classes = test['classe'].unique()
print(classes)

[ 0 -1  1]


## Naive Bayes

In [91]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, confusion_matrix, classification_report

def Naive_Bayes(X_train, y_train, X_val, y_val, X_test, alphas=[0.01, 0.08, 0.09, 0.1, 0.2, 0.5, 0.8, 1.0, 3.0,5.0]):
    best_alpha = None
    best_f1 = 0

    # Procurar o melhor alpha com o conjunto de validação
    for alpha in alphas:
        clf = MultinomialNB(alpha=alpha)
        clf.fit(X_train, y_train)
        y_val_pred = clf.predict(X_val)
        f1 = f1_score(y_val, y_val_pred, average='weighted')
        if f1 > best_f1:
            best_f1 = f1
            best_alpha = alpha
    
    print(f"Melhor alpha encontrado: {best_alpha} (F1 val = {best_f1:.4f})")

    # Treinar o modelo final 
    final_clf = MultinomialNB(alpha=best_alpha)
    final_clf.fit(X_train, y_train)

    y_test_pred = final_clf.predict(X_test)

    return y_test_pred 


Count Vectorizer

In [92]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_NB= Naive_Bayes(X_train_count_vec, y_train, X_val_count_vec,y_val , X_test_count_vec, alphas=[0.01, 0.1, 0.5, 1.0, 5.0]) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_NB, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_NB))

#TF-IDF

#Word2vec

#Glove


----COUNT VECTORIZER ----
Melhor alpha encontrado: 1.0 (F1 val = 0.6949)
              precision    recall  f1-score   support

           1       0.58      0.45      0.51       982
           0       0.76      0.88      0.82      2957
          -1       0.29      0.13      0.18       395

    accuracy                           0.71      4334
   macro avg       0.54      0.48      0.50      4334
weighted avg       0.68      0.71      0.69      4334

[[ 438  524   20]
 [ 256 2592  109]
 [  57  286   52]]


## SVM

In [99]:
def svm():
    import optuna
from sklearn.svm import SVC
from sklearn.metrics import recall_score, classification_report, confusion_matrix

def objective(trial, X_train, X_val, y_train, y_val):

    #Definir hiperparâmetros a testar 
    C = trial.suggest_float('C', 0.01, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3

    #Criar modelo SVM 
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)

    #treinar modelo com conjunto de treino
    model.fit(X_train, y_train)

    #prever no conjunto de validação
    y_pred = model.predict(X_val)

    #calcular f1
    f1 = f1_score(y_val, y_pred, average='macro')

    return f1

def svm(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    # Chamar o Optuna para otimizar os parâmetros
    objeto_para_otimizar = optuna.create_study(direction='maximize')
    objeto_para_otimizar.optimize(lambda trial: objective(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False )

    print("Melhores hiperparâmetros encontrados:")
    print(objeto_para_otimizar.best_params)

    # Treinar o melhor modelo
    best_model = SVC(**objeto_para_otimizar.best_params)
    best_model.fit(X_train, y_train)

    #  Avaliar no conjunto de teste
    y_pred_test = best_model.predict(X_test)

    return y_pred_test

In [100]:
print ("----COUNT VECTORIZER ----")
count_vec_predicted_SVM= svm(X_train_count_vec, X_val_count_vec, X_test_count_vec, y_train, y_val, n_trials=30) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_SVM, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_SVM))

----COUNT VECTORIZER ----
Melhores hiperparâmetros encontrados:
{'C': 3.4224537518113354, 'kernel': 'linear', 'gamma': 'scale'}
              precision    recall  f1-score   support

           1       0.45      0.50      0.47       982
           0       0.77      0.75      0.76      2957
          -1       0.16      0.15      0.16       395

    accuracy                           0.64      4334
   macro avg       0.46      0.46      0.46      4334
weighted avg       0.64      0.64      0.64      4334

[[ 487  432   63]
 [ 498 2216  243]
 [  92  244   59]]


## KNN

In [118]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def KNN(X_train, y_train, X_val, y_val, X_test):
    k_values = range(1, 200,10)  # Determinar  intervalo de valores a testar para k
    best_k = 1
    best_accuracy = 0
    
    k_list = []
    accuracy_list = [] 

    for k in k_values:

        knn = KNeighborsClassifier(n_neighbors=k)  
        knn.fit(X_train, y_train)

        y_pred_KNN = knn.predict(X_val) 
        accuracy = accuracy_score(y_val, y_pred_KNN)  

        k_list.append(k)
        accuracy_list.append(accuracy)  

        if accuracy > best_accuracy: 
            best_accuracy = accuracy
            best_k = k

    knn = KNeighborsClassifier(n_neighbors=best_k)  
    knn.fit(X_train, y_train)

    y_pred_KNN= knn.predict(X_test) 
    print(f"Best k: {best_k}")

    return y_pred_KNN


In [119]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_KNN= KNN(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
lista_y = [str(c) for c in (train['classe'].unique())]
print(metrics.classification_report(
y_test, count_vec_predicted_KNN, target_names=set(lista_y)))
print(metrics.confusion_matrix(y_test, count_vec_predicted_KNN))

#TF-IDF

#Word2vec

#Glove


----COUNT VECTORIZER ----
Best k: 71
              precision    recall  f1-score   support

           1       0.37      0.09      0.15       982
           0       0.69      0.95      0.80      2957
          -1       0.00      0.00      0.00       395

    accuracy                           0.67      4334
   macro avg       0.35      0.35      0.32      4334
weighted avg       0.55      0.67      0.58      4334

[[  91  891    0]
 [ 135 2822    0]
 [  20  375    0]]


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
